In [1]:
import pandas as pd
import datetime as dt
from datetime import date
import numpy as np
import re
from config import datapath, proj_sheet

today = "09042024"

today = date.today() 
week_ago = today - dt.timedelta(days=7)
today = today.strftime("%d%m%Y")
today = "09042024"

datapath = f"/Users/leonahammelrath/FU_Psychoinformatik/Github/tiki_code/data/export_tiki_{today}"
filepath = "/Users/leonahammelrath/FU_Psychoinformatik/Github/tiki_code/data/"

In [2]:
df_active = pd.read_csv(f"https://docs.google.com/spreadsheets/d/{proj_sheet}/export?format=csv")
df_active.rename(columns = {"Pseudonym": "customer", "EMA ID": "ema_id", "Ende EMA Baseline": "end_ema", "Status": "status",
                            "Start EMA Baseline": "start_ema", "Studienversion":"study_version", "FOR_ID":"for_id"}, inplace=True)

#df_active = df_active[["customer", "ema_id", "end_ema", "start_ema", "study_version", "for_id", "status"]]
df_active = df_active[["customer","for_id","start_ema", "end_ema", "status"]]
df_active["customer"] = df_active.customer.str[:4]

In [3]:
answers = pd.read_csv(datapath + "/answers.csv")


/var/folders/ys/gx19nmhj6v30dlkkh5nfp5xh0000gn/T/ipykernel_16020/3116357787.py:1: DtypeWarning: Columns (9) have mixed types. Specify dtype option on import or set low_memory=False.
  answers = pd.read_csv(datapath + "/answers.csv")


In [4]:
answers = pd.read_csv(datapath + "/answers.csv")
answers["user"] = answers["user"].str[:4]
answers = answers[["user", "questionnaireSession", "questionnaire", "study", 
                   "question", "order","element", "createdAt"]]
answers["createdAt"] = (pd.to_datetime(answers["createdAt"],unit='ms'))
answers.rename(columns={"user":"customer","questionnaireSession":"session_unique"}, inplace=True)

/var/folders/ys/gx19nmhj6v30dlkkh5nfp5xh0000gn/T/ipykernel_16020/3105013557.py:1: DtypeWarning: Columns (9) have mixed types. Specify dtype option on import or set low_memory=False.
  answers = pd.read_csv(datapath + "/answers.csv")


In [5]:
choice = pd.read_csv(datapath + "/choice.csv")
choice = choice[["element", "choice_id", "text", "question"]]
choice.rename(columns={"text":"choice_text"}, inplace=True)

In [6]:
questions = pd.read_csv(datapath + "/questions.csv")
questions = questions[["id", "title"]]
questions.rename(columns={"id":"question","title":"quest_title"}, inplace=True)

In [7]:
questionnaire = pd.read_csv(datapath + "/questionnaires.csv")
questionnaire = questionnaire[["id", "name"]]
questionnaire.rename(columns={"id":"questionnaire","name":"questionnaire_name"}, inplace=True)

In [8]:
questionnaire.head()

,questionnaire,questionnaire_name
0,55,TIKI_1A_L1
1,56,TIKI_1A_E1
2,57,TIKI_1M_L1
3,58,TIKI_1M_E1
4,59,TIKI_1E_L2


In [9]:
answer_merged = pd.merge(answers, choice, on= ["question","element"])
answer_merged = pd.merge(answer_merged, questions, on= "question")
answer_merged = pd.merge(answer_merged, questionnaire, on= "questionnaire")

In [10]:
answer_merged = pd.merge(df_active, answer_merged, on="customer")

In [11]:
answer_merged['weekday'] = answer_merged['createdAt'].dt.day_name()

In [12]:
answer_merged['quest_nr'] = answer_merged['questionnaire_name'].apply(lambda x: int(re.search(r'\d+', x).group()) \
                                               if re.search(r'\d+', x) else None)

In [13]:
answer_merged = answer_merged.loc[answer_merged.study.isin([24,25])]

In [18]:
answer_merged = answer_merged.loc[answer_merged.question.isin([340, 352])]

In [19]:
# Sort by UserID and Date
answer_merged = answer_merged.sort_values(by=['customer', 'createdAt'])

# Calculate the day of assessment
answer_merged['assessment_day'] = answer_merged.groupby('customer')['createdAt'].transform(lambda x: (x - x.min()).dt.days + 1)


In [20]:
answer_merged['assessment_id'] = answer_merged.\
apply(lambda row: f"{row['assessment_day']}_{row['quest_nr']}", axis=1)
